# Initialization

In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import datetime

In [2]:
DATA_DIRECTORY = 'your/data/directory' # where you store the raw data

"""
The raw equity returns data used in this project had daily frequency.
One can either keep the daily frequency or go coarser with a weekly frequency
by commenting the proper DATA_AGG string.
Beware that this affects the frequency of the "equity input" described in the thesis
and has no effect on the "financial input" that has a quarterly frequency
"""
# DATA_AGG = 'WEEKLY'
DATA_AGG = 'DAILY'

# comes from previous iterations of the notebook, should be removed soon
OBJECTIVE = 'OPTIMIZATION'

# the model optimizes portfolio composed of the top-N marketcap US equities
N = 500

# Equity prices

In [3]:
# creates the weekly aggregation (if requested) otherwise it sorts the data
def create_weekly_aggregation_equity_prices(equity_prices):
    if DATA_AGG == 'WEEKLY':
        # Create first_day_of_the_week and sort
        equity_prices['first_day_of_the_week'] = equity_prices['date'].dt.to_period('W').dt.start_time
        equity_prices = equity_prices.sort_values(by=['date'], ascending=True).reset_index(drop=True)

        # Create weekly aggregated dataframe
        equity_prices = equity_prices.groupby(['ticker','first_day_of_the_week']).agg({'closeadj': 'last'}).reset_index(drop=False)
        equity_prices = equity_prices.sort_values(by=['first_day_of_the_week'], ascending=True).reset_index(drop=True)

    elif DATA_AGG == 'DAILY':
        equity_prices = equity_prices.sort_values(by=['date'], ascending=True).reset_index(drop=True)

    return equity_prices

In [4]:
# computes logarithimic equity returns
def compute_log_return(equity_prices):
    equity_prices['closeadj_lag1'] = equity_prices.groupby(['ticker'])['closeadj'].shift(1)
    equity_prices['log_return'] = np.log(equity_prices.closeadj/equity_prices.closeadj_lag1)

    return equity_prices

In [5]:
# computes the β-adjusted log returns by subtracting the index (S&P 500) log return which is added with the proper function
def add_beta_adjusted(equity_prices, fund_prices):
    if DATA_AGG == 'WEEKLY':
        left_date = 'first_day_of_the_week'
    elif DATA_AGG == 'DAILY':
        left_date = 'date'
    
    # Adding S&P500 returns
    equity_prices = equity_prices.sort_values(by=left_date, ascending=True).reset_index(drop=True)
    equity_prices['index_log_return'] = pd.merge_asof(left=equity_prices[[left_date,'ticker']], right=fund_prices, on=left_date)['index_log_return']
    equity_prices['log_return_beta_adj'] = equity_prices.log_return - equity_prices.index_log_return

    return equity_prices

In [6]:
# merges data from TTM financials (marketcap, issue date and dividend-adjusted stock price) to the equity returns dataframe
def add_market_cap(equity_prices, financials):
    if DATA_AGG == 'WEEKLY':
        left_date = 'first_day_of_the_week'
    elif DATA_AGG == 'DAILY':
        left_date = 'date'
        
    equity_prices.sort_values(by=left_date, inplace=True)
    equity_prices.reset_index(drop=True, inplace=True)
    financials.sort_values(by='datekey', inplace=True)
    financials.reset_index(drop=True, inplace=True)
    
    equity_prices[['marketcap','datekey','price']] = pd.merge_asof(left=equity_prices[[left_date,'ticker']], right=financials[financials.dimension=='ART'],
                                        left_on=left_date, right_on='datekey', by='ticker')[['marketcap','datekey','price']]
    
    return equity_prices

In [8]:
# from the ARQ financial dataframe, computes the FCFE and creates the 4 features described in the thesis
def add_FCFE(df, financials):
    if DATA_AGG == 'WEEKLY':
        left_date = 'first_day_of_the_week'
    elif DATA_AGG == 'DAILY':
        left_date = 'date'

    financials['working_capital'] = financials['assetsc'] - financials['liabilitiesc']
    financials['working_capital_lag1'] = financials.groupby('ticker')['working_capital'].shift(1)
    financials['change_in_working_capital'] = (financials['working_capital'] - financials['working_capital_lag1']).fillna(0)
    financials['FCFE'] = financials['netinc'] + financials['depamor'] + financials['capex'] - financials['change_in_working_capital'] + financials['ncfdebt']
        
    df.sort_values(by=left_date, inplace=True)
    df.reset_index(drop=True, inplace=True)
    financials.sort_values(by='datekey', inplace=True)
    financials.reset_index(drop=True, inplace=True)

    financials[['FCFE_lag1','revenue_lag1','ncfo_lag1']] = financials.groupby('ticker')[['FCFE','revenue','ncfo']].shift(1)
    financials['change_in_fcfe'] = (financials['FCFE'] - financials['FCFE_lag1']) / (financials['FCFE'].abs())
    financials['change_in_revenue'] = (financials['revenue'] - financials['revenue_lag1']) / (financials['revenue'].abs())
    financials['change_in_ncfo'] = (financials['ncfo'] - financials['ncfo_lag1']) / (financials['ncfo'].abs())
    financials['sign_fcfe'] = financials['FCFE'].apply(np.sign)
    
    float64_cols = list(financials.select_dtypes(include='float64'))
    financials[float64_cols] = financials[float64_cols].astype('float32')

    df[features_fin] = pd.merge_asof(left=df[[left_date,'ticker']], right=financials[financials.dimension=='ARQ'][['ticker','datekey']+features_fin],
                                     left_on=left_date, right_on='datekey', by='ticker')[features_fin]
    print('Added change in FCFE, revenues and NCFO!')
    return df

In [9]:
# updates the marketcap with the current dividend-adjusted (close) price
def update_marketcap(df):
    df['marketcap_updated'] = df['marketcap'] * df['close'] / df['price']

    return df

In [10]:
# creates fut1 entries
def create_beta_adj_fut1(equity_prices):
    equity_prices['log_return_beta_adj_fut1'] = equity_prices.groupby('ticker')['log_return_beta_adj'].shift(-1)
    equity_prices['log_return_fut1'] = equity_prices.groupby('ticker')['log_return'].shift(-1)
    equity_prices['index_log_return_fut1'] = equity_prices.groupby('ticker')['index_log_return'].shift(-1)

    equity_prices.dropna(subset=['log_return_beta_adj_fut1'], inplace=True)
    equity_prices.dropna(subset=['log_return_fut1'], inplace=True)
    equity_prices.dropna(subset=['index_log_return_fut1'], inplace=True)
    equity_prices.dropna(subset=['marketcap_updated'], inplace=True)
    equity_prices.dropna(subset=['change_in_fcfe'], inplace=True)
    equity_prices.dropna(subset=['change_in_revenue'], inplace=True)
    equity_prices.dropna(subset=['change_in_ncfo'], inplace=True)
    equity_prices.dropna(subset=['sign_fcfe'], inplace=True)
    equity_prices.reset_index(drop=True, inplace=True)

    return equity_prices

# Fund prices (S&P500)

In [11]:
# extracts data for the reference S&P 500 index (^GSPC)
def get_fund_prices():
    # Open S&P500 data
    fund_prices = pd.read_csv(f'{DATA_DIRECTORY}/fund_prices/SHARADAR_SFP_2_fb4f5d2244276f3cfeca03f46b122d99.csv',
                                 usecols=['closeadj','ticker','date'], parse_dates=['date'], dtype={'closeadj': 'float32'})
    # Sorting
    fund_prices = fund_prices.sort_values(by='date', ascending=True).reset_index(drop=True)

    if DATA_AGG == 'WEEKLY':
        # Adds weekly time series and sort accordingly
        fund_prices['first_day_of_the_week'] = fund_prices['date'].dt.to_period('W').dt.start_time
        date = 'first_day_of_the_week'
        lag = 5
    elif DATA_AGG == 'DAILY':
        date = 'date'
        lag = 1
    
    # Create returns
    fund_prices[f'closeadj_lag{lag}'] = fund_prices.groupby(['ticker'])['closeadj'].shift(lag)
    fund_prices['index_log_return'] = np.log(fund_prices.closeadj/fund_prices[f'closeadj_lag{lag}'])

    fund_prices = fund_prices[fund_prices.ticker=='^GSPC']
    fund_prices = fund_prices.sort_values(by=date, ascending=True).reset_index(drop=True)

    return fund_prices

# Filtering & selecting

In [18]:
# adds yearly US bond rates to the dataframe (ended up being unused in the thesis)
def is_invalid_float(x):
    try:
        float(x)
        return False
    except ValueError:
        return True

def add_bond_rates(df):
    df.sort_values(by=['date'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    os.chdir(f'{DATA_DIRECTORY}/')
    bonds = pd.read_csv('DGS10.csv', sep=';')
    bonds.rename(columns={'DATE': 'date'}, inplace=True)
    bonds['date'] = pd.to_datetime(bonds['date'], dayfirst=True)
    bonds.sort_values(by='date', inplace=True)
    bonds.reset_index(drop=True, inplace=True)

    # Clean and prepare data for conversion
    bonds['DGS10'] = bonds['DGS10'].str.replace('^\.$', '', regex=True)  # Remove isolated dots

    # Convert to float with error handling
    bonds['DGS10'] = pd.to_numeric(bonds['DGS10'], errors='coerce')
    bonds.dropna(subset='DGS10', inplace=True)
    bonds['DGS10'] = bonds['DGS10'].astype(np.float32)

    df['bonds_yearly_rate'] = pd.merge_asof(left=df['date'], right=bonds, on='date')['DGS10']
    df['bonds_yearly_rate'] = df['bonds_yearly_rate']/100

    return df

In [20]:
# flags data with sufficient past & future LAG history to create the matrices later 
def filter_enough_data(df, LAG, LAG_fin, N_fut):
    # Step 1: Sort DataFrame by 'ticker' and 'date'
    df = df.sort_values(by=['ticker', 'date']).reset_index(drop=True)

    # Step 2: Group by 'ticker'
    grouped = df.groupby('ticker')

    # Initialize an empty list to store the results
    valid_list = []

    # Process each group separately
    for name, group in tqdm(grouped):
        df_tick = group.sort_values(by='datekey').reset_index(drop=True)
        n = len(group)
        
        # Create an array to store valid flags for the group
        valid_flags = [False] * n
        
        for i in range(n):
            # Calculate number of past entries including current
            past_entries = i + 1  # i is 0-based, so i+1 gives the count up to current
            
            # Calculate number of future entries excluding current
            future_entries = n - i - 1  # total length - past - current

            # Calculate for financials features
            datekey_of_index = df_tick.iloc[i]['datekey']
            datekeys = list(df_tick['datekey'].unique())
            idx_dk = datekeys.index(datekey_of_index)
            
            # Check if both conditions are satisfied
            if past_entries >= LAG and future_entries >= N_fut and (idx_dk+1 >= LAG_fin):
                valid_flags[i] = True
        
        # Append the results to the valid_list
        valid_list.extend(valid_flags)

    # Add the result as a new column in the original DataFrame
    df['is_data_valid'] = valid_list

    return df

In [21]:
# ranks each equity by marketcap in each time step
def add_market_ranking(df):
    if DATA_AGG == 'WEEKLY':
        left_date = 'first_day_of_the_week'
    elif DATA_AGG == 'DAILY':
        left_date = 'date'

    df['marketcap_rank_OLD'] = df.groupby(left_date)['marketcap'].rank(ascending=False, method='dense').astype(int)
    df['marketcap_rank'] = np.inf

    """
    to avoid ties, we add a small gaussian noise for the ranking only
    the values of the noise get printed to ensure that it's indeed small
    """
    np.random.seed(0)
    random_noise = np.random.normal(0,1e-3,len(df))
    print(f'Random noise has min={random_noise.min()} and max={random_noise.max()}!')
    df['marketcap_upd_for_ranking'] = df['marketcap_updated'] + random_noise

    df.loc[df['is_data_valid'], 'marketcap_rank'] = df[df['is_data_valid']].groupby(left_date)['marketcap_upd_for_ranking'].rank(ascending=False, method='dense').astype(int)

    if (OBJECTIVE == 'OPTIMIZATION'): # & (not do_i_want_sampling)
        df['marketcap_sum'] = df[df.marketcap_rank <= N].groupby(left_date)['marketcap'].transform('sum')
        df['marketcap_ratio'] = df['marketcap'] / df['marketcap_sum']
        
    return df

# LSTM matrix creation

## Objective: Matrices for Portfolio Optimization

In [16]:
"""
creates the 3D LSTM matrices by handling one ticker at a time and then checking that
the shapes are consistent with the expected one (n_samples, history, n_features*N)
"""
def create_lstm_matrix_optimization(df_dict, ticker_list, date, features, features_fin, target_val, LAG_history, LAG_history_fin, N_fut,
                                    FULL_X, FULL_X_fin, FULL_y, date_flag):
    if DATA_AGG == 'WEEKLY':
        df_date = 'first_day_of_the_week'
    elif DATA_AGG == 'DAILY':
        df_date = 'date'

    X = []
    X_fin = []
    y = []

    if 'marketcap_ratio' in features:
        ind_mcr = features.index('marketcap_ratio')

    for ticker in ticker_list:
        df = df_dict[ticker].reset_index(drop=True)
        idx = df[df[df_date]==date].index[0]

        datekey_of_index = df.iloc[idx]['datekey']
        datekeys = list(df.sort_values(by='datekey')['datekey'].unique())
        idx_dk = datekeys.index(datekey_of_index)

        if (idx+1 >= LAG_history) and (idx < (len(df) - N_fut)) and (idx_dk+1 >= LAG_history_fin):
            # Append past `LAG_history` features to `X`
            X.append(df[features].iloc[idx - LAG_history + 1:idx + 1].values.reshape((1, LAG_history, len(features))))
            X_fin.append(df.loc[df.datekey.isin(datekeys[idx_dk - LAG_history_fin + 1:idx_dk + 1]), features_fin+['datekey']].drop_duplicates()[features_fin].values.reshape((1, LAG_history_fin, len(features_fin))))
            
            # Append future `N_fut` target values to `y`
            # Indices are ok if target_val is NOT fut1
            y.append(df[target_val].iloc[idx + 1:idx + N_fut + 1].values.reshape((1, N_fut, len(target_val))))

    if len(X) > 0:
        X = np.concatenate(X, axis=2)
        X = X.astype(np.float32)
    else:
        X = np.asarray(X)
    if len(y) > 0:
        y = np.concatenate(y, axis=2)
        y = y.astype(np.float32)
    else:
        y = np.asarray(y)
    if len(X_fin) > 0:
        X_fin = np.concatenate(X_fin, axis=2)
        X_fin = X_fin.astype(np.float32)
    else:
        X_fin = np.asarray(X_fin)
    
    if (X.shape == (1, LAG_history, len(features)*len(ticker_list))) & (y.shape == (1, N_fut, len(target_val)*len(ticker_list))) & (X_fin.shape == (1, LAG_history_fin, len(features_fin)*len(ticker_list))):
        # If it's not the correct shape it means that there is some missing data thus we ignore the entry
        if 'marketcap_ratio' in features: # The past history will have the same marketcap_ratio of the considered date
            X[:,:,ind_mcr::len(features)] = X[:,-1,ind_mcr::len(features)] 
            
        date_flag.append(date)
        if len(FULL_X) == 0 & len(FULL_y) == 0:
            FULL_X = np.copy(X)
            FULL_y = np.copy(y)
            FULL_X_fin = np.copy(X_fin)
        else:
            FULL_X = np.concatenate([FULL_X, X], axis=0)
            FULL_y = np.concatenate([FULL_y, y], axis=0)
            FULL_X_fin = np.concatenate([FULL_X_fin, X_fin], axis=0)

    return FULL_X, FULL_X_fin, FULL_y, date_flag

## Saving matrix

In [17]:
# saves the X & y matrices and the support dataframe
def save_matrix(X, X_fin, y, support, sampling=False, date_index=0):
    if OBJECTIVE == 'OPTIMIZATION':
        directory = f'{DATA_DIRECTORY}/LSTM MATRICES/{DATA_AGG}/{OBJECTIVE}/LAG HISTORY {LAG_history} & LAG FIN {LAG_history_fin} & N FUTURES {N_futures}/FEATURES_{features}+{features_fin} - TARGET_{target_val}/TOP-{N} MARKETCAP'
        os.makedirs(directory,exist_ok=True)
        os.chdir(directory)
        np.save(f'TOP-{N}_matrix_X_{DATA_AGG}', X)
        np.save(f'TOP-{N}_matrix_X_fin_{DATA_AGG}', X_fin)
        np.save(f'TOP-{N}_matrix_y_{DATA_AGG}', y)
        support.to_csv(f'TOP-{N}_support_{DATA_AGG}.csv', mode='w', index=False)

## Processing to get matrices

In [38]:
"""
filters the initial dataframe to consider only tickers that are in the top-N marketcap
then creates matrices and support dataframe for each timee step (not a single one due to memory constraints)
"""
def process_for_optimization(df, feature, features_fin, target, LAG, LAG_fin, N_fut, start_date='1800-01-01'):
    if DATA_AGG == 'WEEKLY':
        df_date = 'first_day_of_the_week'
    elif DATA_AGG == 'DAILY':
        df_date = 'date'

    if OBJECTIVE == 'OPTIMIZATION':
        topN_ticker_list = df.loc[df.marketcap_rank <= N, 'ticker'].unique()
        df = df[df['ticker'].isin(topN_ticker_list)].reset_index(drop=True)

        df = df.sort_values(by=[df_date,'marketcap_rank']).reset_index(drop=True)
        dates_list = df.loc[df[df_date]>=start_date, df_date].unique()

        df_dict = {k: v for k,v in df.groupby('ticker')}

        X = []
        X_fin = []
        y = []
        used_dates = []

        for date in tqdm(dates_list):
            tickers_list = df.loc[(df.marketcap_rank <= N) & (df[df_date]==date), 'ticker'].unique()
            X, X_fin, y, used_dates = create_lstm_matrix_optimization(df_dict, tickers_list, date, feature, features_fin, target, LAG, LAG_fin, N_fut,
                                                                      X, X_fin, y, used_dates)
        support = df[(df.marketcap_rank <= N) & (df[df_date].isin(used_dates))]

        save_matrix(X,X_fin,y,support)
    else:
        print('This function is for OPTIMIZATION only!')

## ALL TOGETHER

In [ ]:
"""
select which features to use:
    - features come from data with daily (or weekly) frequency [equity_prices, insiders]
    - features_fin come from data with quarterly frequency [financials]
    - target_val will define the feature contained in the y matrix
"""

features = ['log_return_beta_adj', 'marketcap_ratio']
features_fin = ['change_in_fcfe','change_in_revenue','change_in_ncfo','sign_fcfe']

target_val = ['log_return']

In [22]:
LAG_history = 30 # time series history for features
LAG_history_fin = 8 # time series history for features_fin
N_futures = 10 # time series future history for features_fin

df_name = 'your_merged_dataframe_name.csv' # name of the dataframe that contains all the processed data

### Data processing

#### Create DataFrame

In [ ]:
# loads equity prices datafram
lstm_df = pd.read_csv(f'{DATA_DIRECTORY}/equity_prices/SHARADAR_SEP_2_0afbc06bfa7d2d5ebd28c43e0940ec30.csv',
                               usecols=['date','ticker','closeadj','close'], dtype={'closeadj': 'float32', 'close': 'float32'}, parse_dates=['date'])
print('Loaded equity_prices!')

# loads financials dataframe
financials_df = pd.read_csv(f'{DATA_DIRECTORY}/financials/SHARADAR_SF1_2_1ef2651587ad65788e4bc47f7728edfe.csv',
                            usecols=['datekey','dimension','ticker','shareswa','marketcap','price','netinc','depamor','capex',
                                     'assetsc','liabilitiesc','ncfo','revenue', 'debtc', 'ncfdebt'], parse_dates=['datekey'])
print('Loaded financials!')

# loads the fund prices dataframe
fund_prices_df = get_fund_prices()
print('Loaded fund prices!')

# get minimum and maximum dates (only used for WEEKLY aggregation)
def first_day_of_week(date):
    # Assuming week starts on Monday
    return date - datetime.timedelta(days=date.weekday())

In [ ]:
# process the equity prices, financials and fund prices dataframe to generated the final processed dataframe
lstm_df = create_weekly_aggregation_equity_prices(lstm_df)
lstm_df = compute_log_return(lstm_df)
lstm_df = add_beta_adjusted(lstm_df, fund_prices_df)
del fund_prices_df
lstm_df = add_market_cap(lstm_df, financials_df)
lstm_df = add_FCFE(lstm_df, financials_df)
del financials_df
lstm_df = update_marketcap(lstm_df)
print('Updated marketcap!')
lstm_df = create_beta_adj_fut1(lstm_df)
print('Finished processing equity prices!')

float64_cols = list(lstm_df.select_dtypes(include='float64'))
lstm_df[float64_cols] = lstm_df[float64_cols].astype('float32')
lstm_df.sort_values(by='date', inplace=True)
lstm_df.reset_index(drop=True, inplace=True)

if 'marketcap_ratio' in features:
    assert features.index('marketcap_ratio') == len(features)-1
    fine_features_droppabili = len(features)-1
else:
    fine_features_droppabili = len(features)

lstm_df = add_bond_rates(lstm_df)
print('Added bonds!')

lstm_df[target_val+features[:fine_features_droppabili]+['bonds_yearly_rate']] = lstm_df[target_val+features[:fine_features_droppabili]+['bonds_yearly_rate']].replace([np.inf, -np.inf], np.nan)
lstm_df[features_fin] = lstm_df[features_fin].replace([np.inf, -np.inf], np.nan)

print('Dropping NaNs!')
lstm_df.dropna(subset=target_val+features[:fine_features_droppabili], inplace=True)
lstm_df.dropna(subset=['bonds_yearly_rate'], inplace=True)
lstm_df.dropna(subset=features_fin, inplace=True)
lstm_df.reset_index(drop=True, inplace=True)

lstm_df = filter_enough_data(lstm_df, LAG_history, LAG_history_fin, N_futures)
print('Filtered!')
lstm_df = add_market_ranking(lstm_df)
print('Added marketcap ranking and marketcap_ratio!')
lstm_df.loc[:, 'marketcap_updated'].replace([np.inf, -np.inf], np.nan, inplace=True)
lstm_df.dropna(subset=['marketcap_updated'], inplace=True)

lstm_df.reset_index(drop=True, inplace=True)
lstm_df.drop(columns=['close','closeadj','closeadj_lag1','price'], inplace=True)
print('Finished processing dataframe for LSTM!')

#### Check and save

To save DataFrame for future use

In [27]:
# saves the processed dataframe
os.chdir(f'{DATA_DIRECTORY}/LSTM MATRICES/{DATA_AGG}/{OBJECTIVE}')
lstm_df.to_csv(df_name, mode='w', index=False)

### Loading LSTM DataFrame

In [25]:
# loads the dataframe in float32
os.chdir(f'{DATA_DIRECTORY}/LSTM MATRICES/{DATA_AGG}/{OBJECTIVE}')

if 'marketcap_ratio' in features:
    assert features.index('marketcap_ratio') == len(features)-1
    feat_stop = len(features)-1
else:
    feat_stop = len(features)

datatypes_for_df = {'marketcap': 'float32', 'marketcap_rank': 'float32', 'index_log_return_fut1': 'float32', 'bonds_yearly_rate': 'float32',
                    'index_log_return': 'float32', 'marketcap_updated': 'float32', 'marketcap_ratio': 'float32'}
datatypes_for_df.update({feature: 'float32' for feature in features})
datatypes_for_df.update({feature+'_fut1': 'float32' for feature in features[:feat_stop]})
datatypes_for_df.update({feature_fin: 'float32' for feature_fin in features_fin})
datatypes_for_df.update({target: 'float32' for target in target_val})

if OBJECTIVE == 'OPTIMIZATION':
    if DATA_AGG == 'WEEKLY':
        df_date = 'first_day_of_the_week'
        lstm_df = pd.read_csv(df_name, parse_dates=[df_date],
                        usecols=([df_date, 'ticker', 'index_log_return','marketcap', 'marketcap_rank', 'bonds_yearly_rate', 'index_log_return_fut1', 'datekey',
                                 'relative_shares', 'change_in_portfolio', 'closeadj', 'marketcap_updated', 'datekey'] +
                                 features + [feature+'_fut1' for feature in features[:feat_stop]] + [feature_fin for feature_fin in features_fin] +
                                 [target for target in target_val]),
                        dtype=datatypes_for_df)
    elif DATA_AGG == 'DAILY':
        # loading by chunks because of memory contraints
        df_date = 'date'
        lstm_df = []

        for chunk in tqdm(pd.read_csv(df_name, parse_dates=[df_date],
                        usecols=([df_date, 'ticker', 'index_log_return', 'marketcap', 'marketcap_rank', 'bonds_yearly_rate', 'index_log_return_fut1',
                                  'marketcap_ratio', 'marketcap_updated', 'datekey'] + features + [feature+'_fut1' for feature in features[:feat_stop]] +
                                  [feature_fin for feature_fin in features_fin] + [target for target in target_val]),
                        dtype=datatypes_for_df, chunksize=5000000)):
            lstm_df.append(chunk)
        lstm_df = pd.concat(lstm_df)
        lstm_df['datekey'] = pd.to_datetime(lstm_df['datekey'])

5it [01:23, 16.78s/it]


### Processing by chunks (because of memory issues)

In [39]:
# starts the pipeline to create the single timestep X & y matrices and the support dataframes
if OBJECTIVE == 'OPTIMIZATION':
    print(f'Starting creation of top-{N} matrices for optimization (without sampling)!')
    process_for_optimization(lstm_df, features, features_fin, target_val, LAG_history, LAG_history_fin, N_futures, start_date='2010-01-01')

Starting creation of top-500 matrices for optimization (without sampling)!


100%|██████████| 3571/3571 [1:03:07<00:00,  1.06s/it]


#### Saving S&P500 data

In [ ]:
# saves the index data
os.chdir(f'{DATA_DIRECTORY}/LSTM MATRICES/{DATA_AGG}')
sp500 = get_fund_prices()
sp500['index_log_return_fut1'] = sp500['index_log_return'].shift(-1)
sp500.dropna(subset=['index_log_return_fut1'], inplace=True)
sp500.reset_index(drop=True, inplace=True)
sp500.to_csv('S&P500_index_return_DAILY.csv', mode='w', index=False)